# SBTi-Finance Tool - Aggregation of ready made Temperature Scores
This notebook can be used to run portfolio aggregation using files with ready made Temperature Scores from data providers. See notebooks #1 and #4 for more details on portfolio aggregation. 

Please see the [methodology](https://sciencebasedtargets.org/wp-content/uploads/2020/09/Temperature-Rating-Methodology-V1.pdf), [guidance](https://sciencebasedtargets.org/wp-content/uploads/2020/10/Financial-Sector-Science-Based-Targets-Guidance-Pilot-Version.pdf) and the [technical documentation](https://sciencebasedtargets.github.io/SBTi-finance-tool/)  for more details on the different aggregation methods.

See 1_analysis_example (on [Colab](https://colab.research.google.com/github/ScienceBasedTargets/SBTi-finance-tool/blob/main/examples/1_analysis_example.ipynb) or [Github](https://github.com/ScienceBasedTargets/SBTi-finance-tool/blob/main/examples/1_analysis_example.ipynb)) for more in depth example of how to work with Jupyter Notebooks in general and SBTi notebooks in particular. 


## Setting up
First we will set up the imports and load the file with the company data and temperature scores. 


In [ ]:
%pip install sbti-finance-tool

In [ ]:
%load_ext autoreload
%autoreload 2
import SBTi
from SBTi.portfolio_aggregation import PortfolioAggregationMethod
from SBTi.temperature_score import TemperatureScore
from SBTi.interfaces import ETimeFrames, EScope
import pandas as pd

In [ ]:
# Download the dummy data
import urllib.request
import os

if not os.path.isdir("data"):
    os.mkdir("data")
if not os.path.isfile("data/example_data_provider_TS.xlsx"):
    urllib.request.urlretrieve("https://github.com/ScienceBasedTargets/SBTi-finance-tool/raw/main/examples/data/example_data_provider_TS.xlsx", "data/example_data_provider_TS.xlsx")


In [ ]:
# Create a temperature score instance with the desired time frames and scopes
ts = TemperatureScore(time_frames=[ETimeFrames.SHORT, ETimeFrames.MID, ETimeFrames.LONG], 
    scopes=[EScope.S1S2, EScope.S3, EScope.S1S2S3])

### Load the temperature data

In [ ]:
df_portfolio = pd.read_excel("data/example_data_provider_TS.xlsx")

# Change the column names to match the API if they aren't already in snake_case
if 'ISIN' in df_portfolio.columns:
    df_portfolio.rename(columns={'ISIN': 'isin'}, inplace=True)
if 'LEI' in df_portfolio.columns:
    df_portfolio.rename(columns={'LEI': 'lei'}, inplace=True)

# Convert the scope and time_frame columns to enums (EScope and ETimeFrames)
df_portfolio['scope'] = df_portfolio['scope'].apply(lambda x: EScope[x.upper()])
df_portfolio['time_frame'] = df_portfolio['time_frame'].apply(lambda x: ETimeFrames[x.upper()])
if not 'temperature_results' in df_portfolio.columns:
    df_portfolio['temperature_results'] = 0
# Create a dictionary to store the portfolio scores for different aggregation methods
scores_collection = {}

## Calculate the aggregated temperature score
Calculate an aggregated temperature score. This can be done using different aggregation methods. The termperature scores are calculated per time-frame/scope combination.

### WATS
Weighted Average Temperature Score (WATS): Temperature scores are allocated based on portfolio weights.
This method uses the "investment_value" field to be defined in your portfolio data.

In [ ]:
ts.aggregation_method = PortfolioAggregationMethod.WATS
aggregated_scores = ts.aggregate_scores(df_portfolio)
df_wats = pd.DataFrame(aggregated_scores.dict()).applymap(lambda x: round(x['all']['score'], 2))
scores_collection.update({'WATS': df_wats})
df_wats

### TETS
Total emissions weighted temperature score (TETS): Temperature scores are allocated based on historical emission weights using total company emissions. 
In addition to the portfolios "investment value" the TETS method requires company emissions, please refer to [Data Legends - Fundamental Data](https://ofbdabv.github.io/SBTi/Legends.html#fundamental-data) for more details

In [ ]:
ts.aggregation_method = PortfolioAggregationMethod.TETS
aggregated_scores = ts.aggregate_scores(df_portfolio)
df_tets = pd.DataFrame(aggregated_scores.dict()).applymap(lambda x: round(x['all']['score'], 2))
scores_collection.update({'TETS': df_tets})
df_tets

### MOTS
Market Owned emissions weighted temperature score (MOTS): Temperature scores are allocated based on an equity ownership approach.
In addition to the portfolios "investment value" the MOTS method requires company emissions and market cap, please refer to  [Data Legends - Fundamental Data](https://ofbdabv.github.io/SBTi/Legends.html#fundamental-data) for more details

In [ ]:
ts.aggregation_method = PortfolioAggregationMethod.MOTS
aggregated_scores = ts.aggregate_scores(df_portfolio)
df_mots = pd.DataFrame(aggregated_scores.dict()).applymap(lambda x: round(x['all']['score'], 2))
scores_collection.update({'MOTS': df_mots})
df_mots

### EOTS
Enterprise Owned emissions weighted temperature score (EOTS): Temperature scores are allocated based
on an enterprise ownership approach. 
In addition to the portfolios "investment value" the EOTS method requires company emissions and enterprise value, please refer to  [Data Legends - Fundamental Data](https://ofbdabv.github.io/SBTi/Legends.html#fundamental-data) for more details

In [ ]:
ts.aggregation_method = PortfolioAggregationMethod.EOTS
aggregated_scores = ts.aggregate_scores(df_portfolio)
df_eots = pd.DataFrame(aggregated_scores.dict()).applymap(lambda x: round(x['all']['score'], 2))
scores_collection.update({'EOTS': df_eots})
df_eots

### ECOTS
Enterprise Value + Cash emissions weighted temperature score (ECOTS): Temperature scores are allocated based on an enterprise value (EV) plus cash & equivalents ownership approach. 
In addition to the portfolios "investment value" the ECOTS method requires company emissions, company cash equivalents and enterprise value; please refer to  [Data Legends - Fundamental Data](https://sciencebasedtargets.github.io/SBTi-finance-tool/Legends.html#fundamental-data) for more details

In [ ]:
ts.aggregation_method = PortfolioAggregationMethod.ECOTS
aggregated_scores = ts.aggregate_scores(df_portfolio)
df_ecots = pd.DataFrame(aggregated_scores.dict()).applymap(lambda x: round(x['all']['score'], 2))
scores_collection.update({'ECOTS': df_ecots})
df_ecots

### AOTS
Total Assets emissions weighted temperature score (AOTS): Temperature scores are allocated based on a total assets ownership approach. 
In addition to the portfolios "investment value" the AOTS method requires company emissions and company total assets; please refer to  [Data Legends - Fundamental Data](https://sciencebasedtargets.github.io/SBTi-finance-tool/Legends.html#fundamental-data) for more details

In [ ]:
ts.aggregation_method = PortfolioAggregationMethod.AOTS
aggregated_scores = ts.aggregate_scores(df_portfolio)
df_aots = pd.DataFrame(aggregated_scores.dict()).applymap(lambda x: round(x['all']['score'], 2))
scores_collection.update({'AOTS': df_aots})
df_aots

### ROTS
Revenue owned emissions weighted temperature score (ROTS): Temperature scores are allocated based on the share of revenue.
In addition to the portfolios "investment value" the ROTS method requires company emissions and company revenue; please refer to  [Data Legends - Fundamental Data](https://sciencebasedtargets.github.io/SBTi-finance-tool/Legends.html#fundamental-data) for more details

In [ ]:
ts.aggregation_method = PortfolioAggregationMethod.ROTS
aggregated_scores = ts.aggregate_scores(df_portfolio)
df_rots = pd.DataFrame(aggregated_scores.dict()).applymap(lambda x: round(x['all']['score'], 2))
scores_collection.update({'ROTS': df_rots})
df_rots

See below how each aggregation method impact the scores on for each time frame and scope combination

In [ ]:
pd.concat(scores_collection, axis=0)